# 1.0. Baseline Setup

Since this competition has blocked late submissions, this notebook constructs a test dataset based on the latest dataset released by the host. Additionally, it pre-implements the evaluation metric, SMAPE, so that it can be used consistently across all baseline notebooks.

1. Pre-implements the evaluation metric, SMAPE.
2. Constructs and Store a test dataset.

In [1]:
import os, datetime
from tqdm import tqdm

import numpy as np
import pandas as pd

Load Data to figure out the test period

In [2]:
data_dir = '../data/'

df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_revealed_test = pd.read_csv(os.path.join(data_dir, 'revealed_test.csv'))
df_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
df_census = pd.read_csv(os.path.join(data_dir, 'census_starter.csv'))

In [3]:
df_train.info() # 122,265 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [4]:
print(f'unique counties: {df_train.cfips.nunique()}')

unique counties: 3135


In [5]:
# convert first_day_of_month from str to datetime
df_train.first_day_of_month = pd.to_datetime(df_train.first_day_of_month)
df_test.first_day_of_month = pd.to_datetime(df_test.first_day_of_month)
df_revealed_test.first_day_of_month = pd.to_datetime(df_revealed_test.first_day_of_month)

strformat = '%Y-%m'
print(f'train period:		{df_train.first_day_of_month.min().strftime(strformat)} ~ {df_train.first_day_of_month.max().strftime(strformat)}')
print(f'test period: 		{df_test.first_day_of_month.min().strftime(strformat)} ~ {df_test.first_day_of_month.max().strftime(strformat)}')
print(f'revealed period: 	{df_revealed_test.first_day_of_month.min().strftime(strformat)} ~ {df_revealed_test.first_day_of_month.max().strftime(strformat)}')

train period:		2019-08 ~ 2022-10
test period: 		2022-11 ~ 2023-06
revealed period: 	2022-11 ~ 2022-12


According to the data description for this competition,

> `revealed_test.csv` During the submission period, only the most recent month of data will be used for the public leaderboard. Any test set data older than that will be published in `revealed_test.csv`, closely following the usual data release cycle for the microbusiness report. We expect to publish one copy of `revealed_test.csv` in mid February. This file's schema will match train.csv.

- Only the most recent month of data was used for the public leaderboard.
- `revealed_test.csv` was the only hint of test data during the competition, so the competitors likely had no visibility into the 2023 data.
- The private leaderboard was calculated with approximately 88% of the test data (as described in the private leaderboard instructions).

## 1. Pre-implements the evaluation metric, SMAPE.

Technically, [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) cannot be calculated when both the predicted and actual value are 0 (zero). However, in this competition, the host defined SMAPE = 0 in case where the actual and predicted values are both 0.

In [6]:
# define evaluation metric, SMAPE
def SMAPE(actuals, preds):
	smape = 0
	for actual, pred in zip(actuals, preds):
		if (actual==0) and (pred==0): continue
		smape += abs(actual-pred) / (abs(actual) + abs(pred)) * 2 * 100
	return smape / len(actuals)

# Example data
data = {
    'date': pd.date_range(start='2020-01-01', periods=10, freq='D'),
    'actual': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550],
    'forecast': [110, 140, 210, 260, 295, 340, 410, 460, 480, 570]
}
df = pd.DataFrame(data)

SMAPE(df['actual'], df['forecast']) # it's supposed to be 4.21 %

4.211920087419261

## 2. Constructs and Store a test dataset

Since today is 2024, Godaddy already released Microbusiness Density data from Aug 2019 - Jun 2024. Among four countries, we sole need States data from [this link](https://www.godaddy.com/ventureforward/resource/united-states-microbusiness-density/). I manually downloaded this data on web browser, and added `VF_US_MD_CFIPS_Q224_CSV.csv` file to `godaddy/data` directory. This will be the source of test dataset.

Download info
- Microbusiness Density
- Date Range: Aug 2019 - Jun 2024
- download this data with browser to data/
- godaddy_us_data = 'https://www.godaddy.com/ventureforward/resource/united-states-microbusiness-density/vf_us_md_q224/'

In [7]:
# load data
df_godaddy = pd.read_csv(os.path.join(data_dir, 'VF_US_MD_CFIPS_Q224_CSV.csv'))
df_godaddy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3415 entries, 0 to 3414
Columns: 137 entries, cfips to new_md_jun24
dtypes: float64(134), object(3)
memory usage: 3.6+ MB


### +) Let's compare the revealed dataset on this competition and the dataset on the official website

Column formats
- `pop_18_over_{YYYY}`: population of specific year based on Census.us, ex) `pop_18_over_2022`
- `active_{MON}{YY}`: the number of active microbusiness of specific year and month, ex) `active_nov22`
- `md_{MON}{YY}`: microbusiness density of specific year and month, ex) `md_nov22`

In [8]:
md_nov22_divby_pop22 = (df_godaddy['active_nov22'] / df_godaddy['pop_18_over_2022'] * 100).rename('md_nov22_divby_pop22')
md_nov22_divby_pop21 = (df_godaddy['active_nov22'] / df_godaddy['pop_18_over_2021'] * 100).rename('md_nov22_divby_pop21')
pd.concat([df_godaddy.md_nov22, md_nov22_divby_pop22, md_nov22_divby_pop21], axis=1).head(8)

,md_nov22,md_nov22_divby_pop22,md_nov22_divby_pop21
0,3.292227,3.251472,3.292227
1,7.941944,7.686497,7.941944
2,1.235309,1.252155,1.235309
3,1.275281,1.294480,1.275281
4,1.803057,1.790697,1.803057
5,0.989732,0.981234,0.989732
6,2.261069,2.271648,2.261069
7,2.739139,2.722716,2.739139


From the results above, we can assume that `md_nov22` was calculated using `active_nov22` and `pop_18_over_2021`.

Now, let’s examine other months in 2022.

In [9]:
md_jan22_divby_pop22 = (df_godaddy['active_jan22'] / df_godaddy['pop_18_over_2022'] * 100).rename('md_jan22_divby_pop22')
md_jan22_divby_pop21 = (df_godaddy['active_jan22'] / df_godaddy['pop_18_over_2021'] * 100).rename('md_jan22_divby_pop21')
pd.concat([df_godaddy.md_jan22, md_jan22_divby_pop22, md_jan22_divby_pop21], axis=1).head(8)

,md_jan22,md_jan22_divby_pop22,md_jan22_divby_pop21
0,3.152707,3.113679,3.152707
1,7.437747,7.198518,7.437747
2,1.175294,1.191321,1.175294
3,1.213483,1.231752,1.213483
4,1.721201,1.709402,1.721201
5,0.828900,0.821783,0.828900
6,2.045086,2.054655,2.045086
7,2.657833,2.641897,2.657833


It appears that my assumption is true. 

According to the competition host, the `microbusiness_density` was calcuated using the population lagged by 2 years. If so, the test data from this competition and the data on the official website won't match along. Let's figure it out.

In [10]:
df = df_revealed_test[df_revealed_test.first_day_of_month.dt.month == 11]
df = df.set_index('cfips')['microbusiness_density'].rename('competition_md_nov22')

md_nov22_divby_pop20 = (df_godaddy['active_nov22'] / df_godaddy['pop_18_over_2020'] * 100).rename('md_nov22_divby_pop20')
md_nov22_godaddy = pd.concat([df_godaddy[['cfips', 'md_nov22']], md_nov22_divby_pop20], axis=1).set_index('cfips')

pd.merge(df, md_nov22_godaddy, left_index=True, right_index=True, how='left').head()

,competition_md_nov22,md_nov22,md_nov22_divby_pop20
cfips,,,
1001.0,3.442677,3.292227,3.442677
1003.0,8.257636,7.941944,8.257636
1005.0,1.247223,1.235309,1.247223
1007.0,1.275998,1.275281,1.275998
1009.0,1.836289,1.803057,1.836289


No wonder they doesn't match at all. Instead, the competition values matches with `md_nov22_divby_pop20`.

As a result, the microbusiness density on the official website uses the population over 18, lagged by 1 year, while this competition uses the population over 18, lagged by 2 years. To make my approach fully consistent with the competition’s data, a slight adjustment is needed: divide `active` by the population lagged by 2 years.

In [11]:
# select target data: 2022-11 ~ 2023-06
active_cols = [
	'active_nov22', 'active_dec22', 'active_jan23', 'active_feb23', 'active_mar23', 'active_apr23', 'active_may23', 'active_jun23',
]
pop_cols = ['pop_18_over_2020', 'pop_18_over_2021']

# filter valid cfips and columns
df_godaddy = df_godaddy[df_godaddy.cfips.isin(df_test.cfips)]
df_godaddy = df_godaddy[['cfips'] + active_cols + pop_cols]

In [12]:
# calculate md by the population lagged 2 years
df_godaddy['md_nov22'] = df_godaddy['active_nov22'] / df_godaddy['pop_18_over_2020'] * 100
df_godaddy['md_dec22'] = df_godaddy['active_dec22'] / df_godaddy['pop_18_over_2020'] * 100
df_godaddy['md_jan23'] = df_godaddy['active_jan23'] / df_godaddy['pop_18_over_2021'] * 100
df_godaddy['md_feb23'] = df_godaddy['active_feb23'] / df_godaddy['pop_18_over_2021'] * 100
df_godaddy['md_mar23'] = df_godaddy['active_mar23'] / df_godaddy['pop_18_over_2021'] * 100
df_godaddy['md_apr23'] = df_godaddy['active_apr23'] / df_godaddy['pop_18_over_2021'] * 100
df_godaddy['md_may23'] = df_godaddy['active_may23'] / df_godaddy['pop_18_over_2021'] * 100
df_godaddy['md_jun23'] = df_godaddy['active_jun23'] / df_godaddy['pop_18_over_2021'] * 100

df_godaddy = df_godaddy.drop(columns=active_cols+pop_cols)

# melt columns into a column
df_godaddy = df_godaddy.melt(id_vars=['cfips'], var_name='month', value_name='microbusiness_density')

df_godaddy.head()

,cfips,month,microbusiness_density
0,1001.0,md_nov22,3.442677
1,1003.0,md_nov22,8.257636
2,1005.0,md_nov22,1.247223
3,1007.0,md_nov22,1.275998
4,1009.0,md_nov22,1.836289


In [13]:
print(df_godaddy[df_godaddy.cfips.isin([2195, 15005, 2275])].shape)
df_godaddy[df_godaddy.cfips.isin([2195, 15005, 2275])].head()
# df_revealed_test[df_revealed_test.cfips.isin([2195, 15005, 2275])]

(24, 3)


,cfips,month,microbusiness_density
86,2195.0,md_nov22,NaN
91,2275.0,md_nov22,NaN
546,15005.0,md_nov22,NaN
3221,2195.0,md_dec22,NaN
3226,2275.0,md_dec22,NaN


These three counties have values for active but not for population, yet the competition dataset somehow includes density values for them. Consequently, the recovered dataset contains 24 NaN values, which must be excluded during evaluation.

In [14]:
# map column names with training data
target_cols = ['md_'+col.split('_')[1] for col in active_cols]
target_cols_num = [
	'2022-11-01', '2022-12-01', 
	'2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01'
]

raw2num = {col:colnum for col, colnum in zip(target_cols, target_cols_num)}
df_godaddy.month = df_godaddy.month.map(raw2num)

In [15]:
# generate row_id column
df_godaddy['row_id'] = df_godaddy.cfips.astype(int).astype(str) + '_' + df_godaddy.month
df_godaddy = df_godaddy[['row_id', 'microbusiness_density']]

# sort new test data along the order of submission
sortmap = {val:idx for idx, val in enumerate(df_submission.row_id.values)}
df_godaddy['sortmap'] = df_godaddy.row_id.map(sortmap)
df_godaddy = df_godaddy.sort_values('sortmap').drop(columns='sortmap')

In [16]:
# store data
df_godaddy.to_csv('../data/all_revealed_test.csv', index=False)

Demo Evaluation with submission file and test file

In [17]:
# dropna for evaluation
test = df_godaddy[~df_godaddy.microbusiness_density.isna()]
submission = df_submission[~df_godaddy.microbusiness_density.isna()]

# evaluate
smape_value = SMAPE(test.microbusiness_density.values, submission.microbusiness_density.values).item()
smape_value

59.84950786877098

FYI, private LB (88% of the test data)
- 1st place: 3.7877
- 10st place: 3.8827
- 354st place: 4.1288

In [18]:
# dropna for evaluation
test = df_revealed_test[~df_revealed_test.cfips.isin([2195, 15005, 2275])]
submission = df_godaddy[~df_godaddy.isna()]
submission = submission[submission.row_id.apply(lambda x: True if x.split('_')[1][:4] == 2022 else False)] # 2022 only

# sort new test data along the order of submission
sortmap = {val:idx for idx, val in enumerate(test.row_id.values)}
submission['sortmap'] = submission.row_id.map(sortmap)
submission = submission.sort_values('sortmap').drop(columns='sortmap')

# evaluation with pre-define smape
public_smape = SMAPE(
	test.microbusiness_density.values, 
	submission.microbusiness_density.values)
print(f'estimated public SMAPE: {public_smape:.4f}')

estimated public SMAPE: 0.0000


SMAPE = 0 between `df_revealed_test` and reconstructed test dataset.